In [1]:
def get_res_(values: list):
    value_cnt = {}  # 将结果用一个字典存储
    # 统计结果
    for value in values:
        # get(value, num)函数的作用是获取字典中value对应的键值, num=0指示初始值大小。
        value_cnt[value] = value_cnt.get(value, 0) + 1

    # 打印输出结果
    print(value_cnt)
    print([key for key in value_cnt.keys()])
    print([value for value in value_cnt.values()])

## 对全部数据集train1和train1补充进行过滤

In [2]:
import csv
import re, collections, math, datetime
import json
import collections

import sklearn
import sklearn.svm as svm
from sklearn import preprocessing
from sklearn.metrics import *

import numpy as np


def getDomainFeature(domain: str):
    feature = [domain.count('.'), len(domain), sum(c.isdigit() for c in domain), domain.count('-')]

    count_c = 0
    special_characters = (':', ';', '#', '!', '%', '~', '+', '_', '?', '=', '&', '[', ']')
    for c in domain:
        if c in special_characters:
            count_c = count_c + 1
    feature.append(count_c)

    s = len(domain)
    dd = collections.defaultdict(int)
    for c in domain:
        dd[c] += 1

    # 字符随机性
    # H(d) = － ∑lg( P( Xi ) ) * P( Xi )
    feature.append(sum(map(lambda value: -(value / s) * math.log2(value / s), dd.values())))

    # 元音字母比例
    feature.append(sum(map(lambda x: dd[x], ('a', 'e', 'i', 'o', 'u'))) / s)

    # 唯一字符数
    feature.append(len(dd))

    return feature


def getIPFeature(ip: str):
    if ':' in ip:
        x = ip.split(':')
        feature = x[0].split('.') + [x[1]]
    else:
        feature = ip.split('.') + [80]
    feature = list(map(lambda x: int(x), feature))
    return feature


if __name__ == "__main__":
    ip_pool = []
    ip_label = []

    domain_pool = []
    domain_label = []
    url_pool = []

    n = 0
    count = 0
    path1 = r"suspect_pool_new.txt"
    for _ in csv.reader(open(path1)):
        try:
            if re.match(r"^\d+\.\d+\.\d+\.\d+(:\d+)?$", _[0]):
                ip_pool.append(getIPFeature(_[0]))
                ip_label.append(int(_[1]))
            else:
                if n > 50000:
                    break
                domain_label.append(int(_[1]))
                domain_pool.append(getDomainFeature(_[0]))
                url_pool.append(_[0])
                if _[1] != '0':
                    count+=1
        except:
            print(_)
        n += 1
        if n % 100000 == 0:
            # break
            print(f"已读取分析{n}个数据")
            
    path2 = "data1add.csv"
    for _ in csv.reader(open(path2)):
        try:
            if re.match(r"^\d+\.\d+\.\d+\.\d+(:\d+)?$", _[0]):
                ip_pool.append(getIPFeature(_[0]))
                ip_label.append(int(_[1]))
            else:
                domain_label.append(int(_[1]))
                domain_pool.append(getDomainFeature(_[0]))
                url_pool.append(_[0])
        except:
            print(_)
            
        n += 1
        if n % 100000 == 0:
            # break
            print(f"已读取分析{n}个数据")
            
    print("读取数据完成")

读取数据完成


In [3]:
# get_res_(ip_pool)
get_res_(ip_label)
# get_res_(domain_pool)
get_res_(domain_label)

{3: 3, 8: 8, 4: 406}
[3, 8, 4]
[3, 8, 406]
{2: 11489, 6: 2599, 9: 557, 10: 35, 11: 56, 1: 14, 5: 2, 0: 35249, 3: 170, 7: 11, 8: 875, 12: 7, 4: 4252}
[2, 6, 9, 10, 11, 1, 5, 0, 3, 7, 8, 12, 4]
[11489, 2599, 557, 35, 56, 14, 2, 35249, 170, 11, 875, 7, 4252]


### 开始XGboost

In [4]:
def xgboost_model(x_train,y_train):
    """用XGBoost进行建模，返回训练好的模型"""
    xgboost_clf = XGBClassifier(min_child_weight=6,max_depth=15,
                                objective='multi:softmax',num_class=13)
    print("-" * 60)
    print("xgboost模型：", xgboost_clf)
    xgboost_clf.fit(x_train, y_train)
    # # 打印重要性指数
    # importance_features_top('xgboost', xgboost_clf, x_train)
    # 保存模型
    xgboost_clf.save_model('xgboost_50_3.json')
    # joblib.dump(xgboost_clf, r'XGBoost_model_v1.0')
    return xgboost_clf

In [5]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb
import matplotlib.pyplot as plt

import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score

# 切分训练集和测试集
train_x, test_x, train_y, test_y = train_test_split(domain_pool,domain_label,test_size=0.2,random_state=7)

# xgboost模型初始化设置
xgboost_clf = XGBClassifier(min_child_weight=6,max_depth=10,scale_pos_weight=5,n_estimators=200,
                                objective='multi:softmax',num_class=13,silent=False)

# 建模
# xgboost_clf = xgboost_model(train_x, train_y)
xgboost_clf.fit(train_x, train_y)
# # 打印重要性指数
# importance_features_top('xgboost', xgboost_clf, x_train)
# 保存模型
xgboost_clf.save_model('xgboost_50-part-new.json')
# 预测
pre_y_test = xgboost_clf.predict(test_x)

[09:06:30] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.



In [6]:
# domain_label_a = np.asarray(domain_label_a,dtype=np.int32)

# domain_pool_a = preprocessing.scale(domain_pool_a)

pred_y = pre_y_test # np.around(ypred)
pred_y = pred_y.astype(int)# domain_svm_model.predict(domain_pool_a)
# pred_y_p = domain_svm_model.predict_proba(domain_pool_a)
print(pred_y[500:600])
# get_res_(pred_y)
label_y = test_y
# print(label_y[:100])

[0 0 2 0 0 0 2 0 2 2 0 0 2 2 0 2 0 0 8 0 0 0 0 0 2 0 2 0 2 4 0 0 0 2 2 0 0
 2 0 4 0 0 2 0 0 0 0 0 0 2 2 2 2 0 0 2 4 2 2 2 0 0 2 2 0 0 0 0 2 0 0 0 2 0
 2 2 0 0 0 4 0 0 0 8 2 2 2 0 0 2 2 0 2 2 6 0 0 2 2 0]


In [7]:
# 评估
now = datetime.datetime.now()
timestamp = datetime.datetime.timestamp(now)

TF = {"timestamp":timestamp,
      "randomSeed":42,
      # "训练集规模":int(domain_size * 0.8),
      # "测试集规模":domain_size - int(domain_size * 0.8),
      # "标准化均值":tuple(domain_pool.mean(axis=0)),
      # "标准化方差":tuple(domain_pool.std(axis=0)),
      "准确率": accuracy_score(label_y, pred_y),
      "f1_micro": f1_score(label_y, pred_y,average="micro"),
      "precision_micro": precision_score(label_y, pred_y,average="micro"),
      "recall_micro": recall_score(label_y, pred_y,average="micro"),
      # "auc_micro": roc_auc_score(label_y, pred_y,average="micro"),
      
      "f1_macro": f1_score(label_y, pred_y,average="macro"),
      "recall_macro": recall_score(label_y, pred_y,average="macro"),
      # "auc_macro": roc_auc_score(label_y, pred_y,average="macro"),
      "precision_macro": precision_score(label_y, pred_y,average="macro"),

      "模型名称": "xgboost2"
      }

with open("log_"+TF["模型名称"]+".json","w") as f:
    b = json.dumps(TF)
    f.write(b)

print(f"测试集\033[0;34m准确率\033[0m为\033[0;31m{TF['准确率'] * 100:.2f}%")
print(f"\033[0m测试集\033[0;34mf1_micro值\033[0m为\033[0;31m{TF['f1_micro'] * 100:.2f}%")
print(f"\033[0m测试集\033[0;34mrecall_micro值\033[0m为\033[0;31m{TF['recall_micro'] * 100:.2f}%")
# print(f"\033[0m测试集\033[0;34mroc值\033[0m为\033[0;31m{TF['auc_micro'] * 100:.2f}%")
print(f"\033[0m测试集\033[0;34mprecision_micro值\033[0m为\033[0;31m{TF['precision_micro'] * 100:.2f}%\033[0m")
print(f"\033[0m测试集\033[0;34mf1_macro值\033[0m为\033[0;31m{TF['f1_macro'] * 100:.2f}%")
print(f"\033[0m测试集\033[0;34mrecall_macro值\033[0m为\033[0;31m{TF['recall_macro'] * 100:.2f}%")
# print(f"\033[0m测试集\033[0;34mroc_macro值\033[0m为\033[0;31m{TF['auc_macro'] * 100:.2f}%")
print(f"\033[0m测试集\033[0;34mprecision_macro值\033[0m为\033[0;31m{TF['precision_macro'] * 100:.2f}%\033[0m")

测试集准确率为74.88%
测试集f1_micro值为74.88%
测试集recall_micro值为74.88%
测试集precision_micro值为74.88%
测试集f1_macro值为24.68%
测试集recall_macro值为22.59%
测试集precision_macro值为38.40%


/root/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
import csv
import re, collections, math, datetime
import json
import collections

import sklearn
import sklearn.svm as svm
from sklearn import preprocessing
from sklearn.metrics import *

import numpy as np


def getDomainFeature(domain: str):
    feature = [domain.count('.'), len(domain), sum(c.isdigit() for c in domain), domain.count('-')]

    count_c = 0
    special_characters = (':', ';', '#', '!', '%', '~', '+', '_', '?', '=', '&', '[', ']')
    for c in domain:
        if c in special_characters:
            count_c = count_c + 1
    feature.append(count_c)

    s = len(domain)
    dd = collections.defaultdict(int)
    for c in domain:
        dd[c] += 1

    # 字符随机性
    # H(d) = － ∑lg( P( Xi ) ) * P( Xi )
    feature.append(sum(map(lambda value: -(value / s) * math.log2(value / s), dd.values())))

    # 元音字母比例
    feature.append(sum(map(lambda x: dd[x], ('a', 'e', 'i', 'o', 'u'))) / s)

    # 唯一字符数
    feature.append(len(dd))

    return feature


def getIPFeature(ip: str):
    if ':' in ip:
        x = ip.split(':')
        feature = x[0].split('.') + [x[1]]
    else:
        feature = ip.split('.') + [80]
    feature = list(map(lambda x: int(x), feature))
    return feature


if __name__ == "__main__":
    ip_pool = []
    ip_label = []

    domain_pool = []
    domain_label = []
    url_pool = []

    n = 0
    count = 0
    path1 = r"url_suspect_test.txt"
    for _ in csv.reader(open(path1)):
        # print(_[0])
        try:
            if re.match(r"^\d+\.\d+\.\d+\.\d+(:\d+)?$", _[0]):
                ip_pool.append(getIPFeature(_[0]))
                # ip_label.append(int(_[1]))
            else:
                # domain_label.append(int(_[1]))
                domain_pool.append(getDomainFeature(_[0]))
                url_pool.append(_[0])
                # if _[1] != '0':
                    # count+=1
        except:
            print(_)
        n += 1
        if n % 100000 == 0:
            # break
            print(f"已读取分析{n}个数据")
            
    # path2 = "data1add.csv"
    # for _ in csv.reader(open(path2)):
    #     try:
    #         if re.match(r"^\d+\.\d+\.\d+\.\d+(:\d+)?$", _[0]):
    #             ip_pool.append(getIPFeature(_[0]))
    #             ip_label.append(int(_[1]))
    #         else:
    #             domain_label.append(int(_[1]))
    #             domain_pool.append(getDomainFeature(_[0]))
    #             url_pool.append(_[0])
    #     except:
    #         print(_)
            
        # n += 1
        # if n % 100000 == 0:
        #     # break
        #     print(f"已读取分析{n}个数据")
            
    print("读取数据完成")

已读取分析100000个数据
读取数据完成


In [11]:
pre_y_test = xgboost_clf.predict(domain_pool)

In [12]:
print(len(pre_y_test))
print(len(domain_pool))
print(len(url_pool))

161381
161381
161381


In [13]:
get_res_(pre_y_test)

{0: 138714, 2: 18945, 9: 382, 4: 2095, 6: 856, 8: 370, 3: 9, 11: 10}
[0, 2, 9, 4, 6, 8, 3, 11]
[138714, 18945, 382, 2095, 856, 370, 9, 10]


In [15]:
with open('test-xgb.csv', 'w', newline='\n') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')#,
                            # quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(len(url_pool)): 
        spamwriter.writerow([url_pool[i],pre_y_test[i]])
    # spamwriter.writerow(['Spam', 'Lovely Spam', 'Wonderful Spam'])